In [1]:
import torch
import torchaudio
import torchaudio.transforms as TAT

import os

In [2]:
z_dim = 4
music_folder = r"C:\Users\dan\Desktop\sound_vae\data\techno_music"

wavs = []
rats = []

for path in os.listdir(music_folder):
    audio_file = os.path.join(music_folder, path)
    if os.path.isfile(audio_file):
        wav, rate = torchaudio.load(audio_file)
        wavs.append(wav)
        rats.append(rate)

In [4]:
means = []
std1s = []

for wav, rate in zip(wavs, rats):
    processor_fft = TAT.MelSpectrogram(
                            sample_rate=rate,
                            n_mels=z_dim,
                            ).to("cuda").double()

    mel = processor_fft(wav.to("cuda"))
    mean = mel.mean(dim=(0,2), keepdim=True)
    std1 = mel.var(dim=(0,2), keepdim=True).sqrt()

    means.append(mean)
    std1s.append(std1)

In [5]:
print(means[0].shape)
means_t = torch.cat(means, 0).mean(0).squeeze()
print(means_t.shape)
print(std1s[0].shape)
std1s_t = torch.cat(std1s, 0).mean(0).squeeze()
print(std1s_t.shape)
print("------------")
print("mean: ", means_t.cpu().numpy().tolist() )
print("std1: ", std1s_t.cpu().numpy().tolist() )

torch.Size([1, 4, 1])
torch.Size([4])
torch.Size([1, 4, 1])
torch.Size([4])
------------
mean:  [452.15845655661803, 70.35665321114695, 42.851305103906, 22.160291749979983]
std1:  [510.373306840422, 135.12631741685448, 102.5477751543778, 63.582527648859525]


In [ ]:
print(mel.shape)
mean = mel.mean(dim=(0,2), keepdim=True)
std3 = mel.var(dim=(0,2), keepdim=True).sqrt()*3
print("mean", mean)
print("min", mel.amin(dim=2))
print("max", mel.amax(dim=2))
print("std3", std3)

normalized = mel - mean
normalized /= std3

print("mean", normalized.mean(dim=(0,2), keepdim=True))
print("min", normalized.amin(dim=2))
print("max", normalized.amax(dim=2))
print("std3", normalized.var(dim=(0,2), keepdim=True).sqrt()*3)
